In [1]:
USE CuatroCuadras

Commands completed successfully.

Total execution time: 00:00:00.095

# Cuatro Cuadras #
## Consultas##

## 1
Qué lugares son los más frecuentados por personas mayores a 45 años

In [8]:
SELECT TOP 3 count(*) AS visitas, L.ID_Lugar, L.Nombre FROM Lugar L
INNER JOIN Visita V  
ON L.ID_Lugar=V.ID_Lugar
INNER JOIN Usuario U 
ON V.Nickname=U.Nickname
WHERE U.Edad>45
GROUP BY L.ID_Lugar, L.Nombre

(3 rows affected)

Total execution time: 00:00:00.057

visitas,ID_Lugar,Nombre
1,5,Estadio Teodoro Mariscal
1,9,Pemex
1,10,Farmacia Guadalajara


## 2
Cuáles son los dos lugares menos visitados

In [11]:
SELECT TOP 2 count(*) AS Visitas, L.ID_Lugar, L.Nombre FROM LUGAR L 
INNER JOIN VISITA V 
ON L.ID_Lugar=V.ID_Lugar
GROUP BY L.ID_Lugar, L.Nombre
ORDER BY Visitas ASC

(2 rows affected)

Total execution time: 00:00:00.069

Visitas,ID_Lugar,Nombre
1,7,KFC Insurgentes
1,15,Banbajío


## 3
Cuáles son los lugares que tienen "muy malo" o "malo" como comentario y ningun "Muy Bueno", "bueno" o "excelente"

In [17]:
SELECT DISTINCT L.ID_Lugar, L.Nombre FROM Lugar L
INNER JOIN VISITA V 
ON L.ID_Lugar=V.ID_Lugar
WHERE v.Comentario LIKE '%malo%' AND V.ID_Lugar NOT IN 
(SELECT DISTINCT ID_Lugar FROM Visita WHERE Comentario LIKE '%bueno%' OR Comentario LIKE '%excelente%' OR Comentario LIKE '%Muy bueno%')


(1 row affected)

Total execution time: 00:00:02.960

ID_Lugar,Nombre
13,USE Navolato


## 4
Cuáles son los 3 lugares qué cuentan con la mejor valoración

In [20]:
WITH Promedio (ID_Lugar, Nombre, Valoracion)
AS
(
    SELECT l.ID_Lugar, L.Nombre, AVG(V.Valoracion) AS ValoracionTotal FROM LUGAR L
    INNER JOIN VISITA V 
    ON L.ID_Lugar=V.ID_Lugar
    GROUP BY L.ID_Lugar, L.Nombre
)
SELECT TOP 3 * FROM Promedio
ORDER BY Valoracion DESC

(3 rows affected)

Total execution time: 00:00:00.091

ID_Lugar,Nombre,Valoracion
21,The Barber Shop,9
9,Pemex,8
10,Farmacia Guadalajara,8


## 5
Qué usuarios han obtenido cuando menos un logro

In [22]:
SELECT * FROM USUARIO U 
WHERE U.Nickname IN (SELECT Nickname FROM LOGRO_USUARIO)

(5 rows affected)

Total execution time: 00:00:01.011

Nickname,Nombre,Apellidos,Sexo,Edad,Email,Contraseña,ID_Ciudad
AnaALV,Ana Maria,Alvarez Castelo,F,26,MarianaALV@hotmail.com,MariAnaLV1582,6
Coronavirus,Angel,Gomez Bravo,M,19,CoronaPandamiaVirus@hotmail.com,AntiVacunas2020,6
Luiz,Luis,Lopez Lopez,M,30,LuisLopezAlCuadrado@hotmail.com,LopezAlCuadrado,1
SandCast,Sandra,Flores Castilla,F,16,SandraCast@hotmail.com,Sandra1235,4
XxXGokuXxX,Juan,Perez Perez,M,14,XxXGokuSSJXxX@hotmail.com,ContraseñaDeLaCIA,4


## 6
Cuáles han sido los usuarios que han hecho visitas en una cidad distina a la de su origen

In [24]:
SELECT DISTINCT U.Nickname AS Usuario FROM USUARIO U 
INNER JOIN VISITA V 
ON U.Nickname=V.Nickname
INNER JOIN LUGAR L 
ON V.ID_Lugar=L.ID_Lugar
WHERE U.ID_Ciudad<>L.ID_Ciudad

(16 rows affected)

Total execution time: 00:00:00.049

Nickname
AlizV5
AnaALV
Angel224
Coronavirus
CuentaFeik
Diana129
Emilian99
GabiLuna
JoseCarl
Luiz


## 7
Qué usuarios han hecho amistad con usuarios de una ciudad distinta a la de ellos

In [38]:
SELECT A.Nickname1 AS [Usuario 1], A.Nickname2 AS [Usuario 2] FROM AMIGO A 
INNER JOIN USUARIO U 
ON A.Nickname1=U.Nickname
INNER JOIN USUARIO U2 
ON A.Nickname2=U2.Nickname
WHERE U.ID_Ciudad<>U2.ID_Ciudad

(16 rows affected)

Total execution time: 00:00:00.048

Usuario 1,Usuario 2
AnaALV,Mari
Angel224,Diana129
Coronavirus,Angel224
Diana129,Mari
Emilian99,AnaALV
GabiLuna,RamLop99
JoseCarl,MenitaXD98
JoseCarl,XxXGokuXxX
Luiz,Coronavirus
Luiz,Mari


## 8
Se requiere saber el lugar más frecuentado por mujeres

In [37]:
WITH Frecuencia(Nombre, Visitas )
AS 
(
    SELECT L.Nombre, count(*) AS Visitas FROM LUGAR L 
    INNER JOIN VISITA V 
    ON L.ID_Lugar=V.ID_Lugar
    INNER JOIN USUARIO U 
    ON V.Nickname=U.Nickname
    WHERE U.Sexo='F'
    GROUP BY L.Nombre
)
SELECT TOP 1 * FROM Frecuencia
ORDER BY Visitas DESC

(1 row affected)

Total execution time: 00:00:00.143

Nombre,Visitas
Estadio Teodoro Mariscal,3


## 9
Conocer todos los usuarios que han comentado con "Muy malo" a sus visitas

In [31]:
SELECT U.Nickname FROM VISITA V 
INNER JOIN USUARIO U 
ON V.Nickname=U.Nickname 
WHERE V.Comentario='Muy malo'

(5 rows affected)

Total execution time: 00:00:00.107

Nickname
Angel224
Luiz
Coronavirus
SandCast
XxXGokuXxX


## 10
Se desea saber cuál es el promedio de valoración de todos los lugares

In [36]:
SELECT L.Nombre,  AVG(V.Valoracion) AS Valoracion FROM LUGAR L 
INNER JOIN VISITA V 
ON L.ID_Lugar=V.ID_Lugar 
GROUP BY L.Nombre

(21 rows affected)

Total execution time: 00:00:00.047

Nombre,Valoracion
Aeropuerto Internacional de Tijuana,7
Banbajío,6
Cinemex Navolato,7
Estadio Teodoro Mariscal,7
Estadio Tomateros,7
Farmacia Guadalajara,8
Fühler,6
Hospital Ángeles,6
KFC Insurgentes,2
Ley Primavera,7
